In [2]:
%pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (37.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2

In [6]:
import os
import cv2
import numpy as np
import random
import shutil
from scipy.ndimage import gaussian_filter
from scipy.fftpack import fft2, ifft2
from skimage.exposure import rescale_intensity
import albumentations as A
from tqdm import tqdm

# Define Paths
INPUT_DIR = "data"
OUTPUT_DIR = "data_preprocessed"

# Ensure output directories exist
for category in ["Ai_Images", "Camera_Images"]:
    os.makedirs(os.path.join(OUTPUT_DIR, category), exist_ok=True)

# Preprocessing Functions
def apply_low_pass_filter(image):
    """Applies a Gaussian Low-Pass Filter to blur high-frequency noise."""
    return cv2.GaussianBlur(image, (5, 5), 0)

def apply_medium_pass_filter(image):
    """Applies a medium-pass filter by subtracting low-pass from original."""
    low_pass = apply_low_pass_filter(image)
    return cv2.addWeighted(image, 1.5, low_pass, -0.5, 0)

def apply_high_pass_filter(image):
    """Applies a High-Pass Filter to enhance edges."""
    kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def apply_high_boost_filter(image, alpha=1.5):
    """Applies High-Boost Filtering to enhance details."""
    low_pass = apply_low_pass_filter(image)
    return cv2.addWeighted(image, alpha, low_pass, -1 * (alpha - 1), 0)

def apply_homomorphic_filter(image):
    """Applies Homomorphic Filtering (Frequency Domain)."""
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_log = np.log1p(np.array(image, dtype="float"))
    fft = fft2(image_log)
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    mask = np.ones((rows, cols), np.float32)
    mask[crow - 30:crow + 30, ccol - 30:ccol + 30] = 0  # Low-pass suppression

    fft_shift = fft * mask
    filtered_image = np.abs(ifft2(fft_shift))
    return rescale_intensity(filtered_image, out_range=(0, 255)).astype("uint8")

def apply_sobel_filter(image):
    """Applies Sobel Edge Detection."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    sobel = cv2.magnitude(sobelx, sobely)
    return cv2.cvtColor(np.uint8(sobel), cv2.COLOR_GRAY2BGR)

def apply_image_erosion(image):
    """Applies Morphological Erosion."""
    kernel = np.ones((3, 3), np.uint8)
    return cv2.erode(image, kernel, iterations=1)

def apply_image_dilation(image):
    """Applies Morphological Dilation."""
    kernel = np.ones((3, 3), np.uint8)
    return cv2.dilate(image, kernel, iterations=1)

# Data Augmentation Pipeline (Albumentations)
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.GridDistortion(p=0.2),
    A.ElasticTransform(p=0.2)
])

# Processing Images
for category in ["Ai_Images", "Camera_Images"]:
    input_path = os.path.join(INPUT_DIR, category)
    output_path = os.path.join(OUTPUT_DIR, category)

    for img_name in tqdm(os.listdir(input_path), desc=f"Processing {category}"):
        img_path = os.path.join(input_path, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue

        # Resize for consistency
        image = cv2.resize(image, (256, 256))

        # Apply preprocessing filters
        low_pass = apply_low_pass_filter(image)
        medium_pass = apply_medium_pass_filter(image)
        high_pass = apply_high_pass_filter(image)
        high_boost = apply_high_boost_filter(image)
        homomorphic = apply_homomorphic_filter(image)
        sobel = apply_sobel_filter(image)
        eroded = apply_image_erosion(image)
        dilated = apply_image_dilation(image)

        # Save Preprocessed Images
        cv2.imwrite(os.path.join(output_path, f"lowpass_{img_name}"), low_pass)
        cv2.imwrite(os.path.join(output_path, f"mediumpass_{img_name}"), medium_pass)
        cv2.imwrite(os.path.join(output_path, f"highpass_{img_name}"), high_pass)
        cv2.imwrite(os.path.join(output_path, f"highboost_{img_name}"), high_boost)
        cv2.imwrite(os.path.join(output_path, f"homomorphic_{img_name}"), homomorphic)
        cv2.imwrite(os.path.join(output_path, f"sobel_{img_name}"), sobel)
        cv2.imwrite(os.path.join(output_path, f"eroded_{img_name}"), eroded)
        cv2.imwrite(os.path.join(output_path, f"dilated_{img_name}"), dilated)

        # Data Augmentation (Save multiple variants)
        for i in range(3):
            augmented = augmentations(image=image)["image"]
            cv2.imwrite(os.path.join(output_path, f"aug_{i}_{img_name}"), augmented)

print(" Preprocessing and Augmentation Complete. Processed images saved in:", OUTPUT_DIR)


Processing Camera_Images: 100%|██████████| 233/233 [00:04<00:00, 50.06it/s]

 Preprocessing and Augmentation Complete. Processed images saved in: data_preprocessed


In [8]:
import os
import cv2
import numpy as np
from scipy.fftpack import fft2, ifft2
from skimage.exposure import rescale_intensity
import albumentations as A
from tqdm import tqdm

# Define Paths
INPUT_DIR = "../archive"  # Change this if needed
OUTPUT_DIR = "../archive_preprocessed"

# Folder Structure (Train/Test -> FAKE/REAL)
DATASETS = ["train", "test"]
CATEGORIES = ["FAKE", "REAL"]

# Ensure output directories exist
for dataset in DATASETS:
    for category in CATEGORIES:
        os.makedirs(os.path.join(OUTPUT_DIR, dataset, category), exist_ok=True)

# Preprocessing Functions
def apply_low_pass_filter(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

def apply_medium_pass_filter(image):
    low_pass = apply_low_pass_filter(image)
    return cv2.addWeighted(image, 1.5, low_pass, -0.5, 0)

def apply_high_pass_filter(image):
    kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def apply_high_boost_filter(image, alpha=1.5):
    low_pass = apply_low_pass_filter(image)
    return cv2.addWeighted(image, alpha, low_pass, -1 * (alpha - 1), 0)

def apply_homomorphic_filter(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_log = np.log1p(np.array(image, dtype="float"))
    fft = fft2(image_log)
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    mask = np.ones((rows, cols), np.float32)
    mask[crow - 30:crow + 30, ccol - 30:ccol + 30] = 0  # Low-pass suppression

    fft_shift = fft * mask
    filtered_image = np.abs(ifft2(fft_shift))
    return rescale_intensity(filtered_image, out_range=(0, 255)).astype("uint8")

def apply_sobel_filter(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    sobel = cv2.magnitude(sobelx, sobely)
    return cv2.cvtColor(np.uint8(sobel), cv2.COLOR_GRAY2BGR)

def apply_image_erosion(image):
    kernel = np.ones((3, 3), np.uint8)
    return cv2.erode(image, kernel, iterations=1)

def apply_image_dilation(image):
    kernel = np.ones((3, 3), np.uint8)
    return cv2.dilate(image, kernel, iterations=1)

# Data Augmentation Pipeline (Albumentations)
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.GridDistortion(p=0.2),
    A.ElasticTransform(p=0.2)
])

# Process Images
for dataset in DATASETS:
    for category in CATEGORIES:
        input_path = os.path.join(INPUT_DIR, dataset, category)
        output_path = os.path.join(OUTPUT_DIR, dataset, category)

        for img_name in tqdm(os.listdir(input_path), desc=f"Processing {dataset}/{category}"):
            img_path = os.path.join(input_path, img_name)
            image = cv2.imread(img_path)

            if image is None:
                continue

            # Resize for consistency
            image = cv2.resize(image, (256, 256))

            # Apply preprocessing filters
            low_pass = apply_low_pass_filter(image)
            medium_pass = apply_medium_pass_filter(image)
            high_pass = apply_high_pass_filter(image)
            high_boost = apply_high_boost_filter(image)
            homomorphic = apply_homomorphic_filter(image)
            sobel = apply_sobel_filter(image)
            eroded = apply_image_erosion(image)
            dilated = apply_image_dilation(image)

            # Save Preprocessed Images
            cv2.imwrite(os.path.join(output_path, f"lowpass_{img_name}"), low_pass)
            cv2.imwrite(os.path.join(output_path, f"mediumpass_{img_name}"), medium_pass)
            cv2.imwrite(os.path.join(output_path, f"highpass_{img_name}"), high_pass)
            cv2.imwrite(os.path.join(output_path, f"highboost_{img_name}"), high_boost)
            cv2.imwrite(os.path.join(output_path, f"homomorphic_{img_name}"), homomorphic)
            cv2.imwrite(os.path.join(output_path, f"sobel_{img_name}"), sobel)
            cv2.imwrite(os.path.join(output_path, f"eroded_{img_name}"), eroded)
            cv2.imwrite(os.path.join(output_path, f"dilated_{img_name}"), dilated)

            # Data Augmentation (Save multiple variants)
            for i in range(3):
                augmented = augmentations(image=image)["image"]
                cv2.imwrite(os.path.join(output_path, f"aug_{i}_{img_name}"), augmented)

print(" Preprocessing and Augmentation Complete. Processed images saved in:", OUTPUT_DIR)


Processing test/REAL: 100%|██████████| 10000/10000 [01:18<00:00, 127.51it/s]

✅ Preprocessing and Augmentation Complete. Processed images saved in: ../archive_preprocessed


In [13]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
from tqdm import tqdm

# Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Paths
DATA_DIR = "../archive_preprocessed"
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)
NUM_CLASSES = 2  # FAKE and REAL
EPOCHS = 10
LEARNING_RATE = 0.0001

# Define Transforms (Data Augmentation)
train_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Datasets
train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_transforms)
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "test"), transform=test_transforms)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Load Pretrained ResNet101 Model
model = models.resnet101(pretrained=True)

# Modify Final Layer for Binary Classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)

# Move Model to Device
model = model.to(device)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training Function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            # Forward Pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward Pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track Loss and Accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total * 100
        print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%")

    print(" Training Complete!")

# Model Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating Model"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total * 100
    print(f" Model Accuracy on Test Set: {accuracy:.2f}%")

    return all_preds, all_labels

# Train and Evaluate Model
train_model(model, train_loader, criterion, optimizer, epochs=EPOCHS)
evaluate_model(model, test_loader)

# Save the Model
MODEL_PATH = "resnet101_fake_vs_real.pth"
torch.save(model.state_dict(), MODEL_PATH)
print(f" Model saved at {MODEL_PATH}")


/Users/amritaajaysagar/miniconda3/envs/image_env/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/amritaajaysagar/miniconda3/envs/image_env/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /Users/amritaajaysagar/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:03<00:00, 50.1MB/s] 
Epoch 1/10:   2%|▏         | 652/34375 [2:15:38<116:55:19, 12.48s/it]


KeyboardInterrupt: 

In [12]:
pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
from tqdm import tqdm

# Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Paths
DATA_DIR = "../archive_preprocessed"
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)
NUM_CLASSES = 2  # FAKE and REAL
EPOCHS = 5  # Reduce epochs for faster training
LEARNING_RATE = 0.0005  # Increase slightly for faster convergence

# Define Transforms (Data Augmentation)
train_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Datasets
train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_transforms)
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "test"), transform=test_transforms)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Load EfficientNet-B0 Model (Lightweight and Fast)
model = models.efficientnet_b0(pretrained=True)

# Modify Final Layer for Binary Classification
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, NUM_CLASSES)

# Move Model to Device
model = model.to(device)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training Function
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            # Forward Pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward Pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track Loss and Accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total * 100
        print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%")

    print("Training Complete!")

# Model Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating Model"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total * 100
    print(f" Model Accuracy on Test Set: {accuracy:.2f}%")

    return all_preds, all_labels

# Train and Evaluate Model
train_model(model, train_loader, criterion, optimizer, epochs=EPOCHS)
evaluate_model(model, test_loader)

# Save the Model
MODEL_PATH = "efficientnet_b0_fake_vs_real.pth"
torch.save(model.state_dict(), MODEL_PATH)
print(f" Model saved at {MODEL_PATH}")
